In [1]:
# imports and setup
%cd '/home/naodell/work/z_plus_jpsi'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
matplotlib.style.use('default')

from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from root_pandas import read_root

import scripts.plot_tools as pt
import scripts.nn_helpers as nnhelper

#%connect_info

/home/naodell/work/z_plus_jpsi


Using TensorFlow backend.


Welcome to JupyROOT 6.11/01


In [2]:
### get the data and prepare labels
disc_features = ['n_jets', 'n_fwdjets', 'n_bjets'] 
cont_features = [
                 'lepton1_pt', 'lepton1_eta', 'lepton1_phi', 'lepton1_iso',              
                 'lepton2_pt', 'lepton2_eta', 'lepton2_phi', 'lepton2_iso',              
                 #'lepton1_d0', 'lepton1_dz',
                 #'lepton2_d0', 'lepton2_dz',
    
                 'dilepton1_delta_eta', 'dilepton1_delta_phi', 'dilepton1_delta_r',
                 'dilepton1_mass', 'dilepton1_pt', 'dilepton1_eta', 'dilepton1_phi',
                 'dilepton1_pt_asym',
                                                                                         
                 'met_mag', 'met_phi', 'ht_mag', 'ht_phi',
                 'jet1_pt', 'jet1_eta', 'jet1_phi', 'jet1_tag',                                     
                 'jet2_pt', 'jet2_eta', 'jet2_phi', 'jet2_tag',                                     
                 'jet_delta_eta', 'jet_delta_phi', 'jet_delta_r',                        
                 'dijet_mass', 'dijet_pt', 'dijet_eta', 'dijet_phi',                     
                                                                                         
                 'lepton1_j1_mass', 'lepton1_j1_pt',
                 'lepton1_j1_delta_r', 'lepton1_j1_delta_eta', 'lepton1_j1_delta_phi',
                 'lepton2_j1_mass', 'lepton2_j1_pt',
                 'lepton2_j1_delta_r', 'lepton2_j1_delta_eta', 'lepton2_j1_delta_phi',
                 'lepton1_j2_mass', 'lepton1_j2_pt',
                 'lepton1_j2_delta_r', 'lepton1_j2_delta_eta', 'lepton1_j2_delta_phi',
                 'lepton2_j2_mass', 'lepton2_j2_pt',
                 'lepton2_j2_delta_r', 'lepton2_j2_delta_eta', 'lepton2_j2_delta_phi',
    
                 'dilepton_j1_mass', 'dilepton_j1_pt',                                   
                 'dilepton_j1_delta_r', 'dilepton_j1_delta_eta', 'dilepton_j1_delta_phi',
                 'dilepton_j2_mass', 'dilepton_j2_pt',                                   
                 'dilepton_j2_delta_r', 'dilepton_j2_delta_eta', 'dilepton_j2_delta_phi',
                 'four_body_mass',                                                       
                 'four_body_delta_r', 'four_body_delta_eta', 'four_body_delta_phi',      
                ]

cuts = 'lepton1_pt > 10 and abs(lepton1_eta) < 2.4 \
        and lepton2_pt > 15 and abs(lepton2_eta) < 2.5 \
        and lepton1_reliso < 0.15 \
        and lepton1_q != lepton2_q \
        and 12 < dilepton1_mass \
        and n_jets + n_bjets >= 2 \
        and weight >= 0' # need to figure out why I have negative weights (no I don't use generator weights)

datasets    = [
               #'muon_2016B', 'muon_2016C', 'muon_2016D',
               #'muon_2016E', 'muon_2016F', 'muon_2016G', 'muon_2016H',
               #'electron_2016B', 'electron_2016C', 'electron_2016D',
               #'electron_2016E', 'electron_2016F', 'electron_2016G', 'electron_2016H',
               'mueg_2016B', 'mueg_2016C', 'mueg_2016D',
               'mueg_2016E', 'mueg_2016F', 'mueg_2016G', 'mueg_2016H',

               'ttbar_lep', 't_tw', 'tbar_tw',             
               #'zjets_m-50',  'zjets_m-10to50',  
               #'data_ss'
               ] 

dm = pt.DataManager(input_dir     = 'data/flatuples/emu_2016',
                    dataset_names = datasets,
                    selection     = 'emu',
                    period        = 2016,
                    scale         = 36.4e3,
                    cuts          = cuts
                   )

df_top = pd.concat([dm.get_dataframe('ttbar'), dm.get_dataframe('t')])
#df_zjets = dm.get_dataframe('zjets')
#df_fakes = dm.get_dataframe('data_ss')
#df_data  = dm.get_dataframe('data')

Loading dataframes:   0%|                      | 0.00/10.0 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
Loading dataframes: 100%|███████████████| 10.0/10.0 [00:04<00:00, 1.32it/s]


KeyError: 'ttbar'

In [ ]:
# data preprocessing

# reweight ss selection to corresond to probability of sampling this event (I don't think this is right)
#n_evt = (df_zjets.weight.sum() + df_top.weight.sum() + df_fakes.weight.sum())
#df_fakes['weight'] *= 1./n_evt

# split top events based on decay mode
df_top.loc[(abs(df_top.lepton1_mother) == 24) & (abs(df_top.lepton2_mother) == 24), 'label'] = 'emu'
df_top.loc[(abs(df_top.lepton1_mother) == 15) & (abs(df_top.lepton2_mother) == 24), 'label'] = 'etau'
df_top.loc[(abs(df_top.lepton1_mother) == 24) & (abs(df_top.lepton2_mother) == 15), 'label'] = 'mutau'
df_top.loc[(abs(df_top.lepton1_mother) == 15) & (abs(df_top.lepton2_mother) == 15), 'label'] = 'tautau'

# combine dataframes, group non-top backgrounds, and drop events where gen matching failed
df_top = df_top.query('label != "ttbar" and label!="t"')
#df_zjets.label = 'bg'
#df_fakes.label = 'bg'
#df_combined = pd.concat([df_top, df_zjets, df_fakes])
df_combined = df_top

# set feature ranges and fill underflow/overflow bins; scale features to lie between 0 and 1; binarize discrete variables
df_trans = nnhelper.preprocess_data(df_combined, cont_features, disc_features, dm.get_bounds_dict())

In [ ]:
# plot the transformed variables
if False:
    df_trans.hist(cont_features, bins=50, histtype='stepfilled', figsize=(25, 25))
    plt.yscale('log')
    plt.show()

In [ ]:
# sample from dataframe according to event weights; split into testing and training sets
n_total   = df_trans.shape[0]
split     = int(0.8*n_total)

df_sample = df_trans.sample(frac=1)#.reset_index()

# training sample
df_train = df_sample[:split] 
x_train = df_train.drop(['label', 'weight'], axis=1).values
y_train = df_train['label'].values

df_test = df_sample[split:]
x_test = df_test.drop(['label', 'weight'], axis=1).values
y_test = df_test['label'].values

# create one hot encoding for category labels
label_enc = preprocessing.LabelBinarizer()
label_enc.fit(y_train)
onehot_enc = preprocessing.OneHotEncoder(sparse=False)
y_train = label_enc.transform(y_train)
y_test  = label_enc.transform(y_test)

In [ ]:
# train the model
#model = nnhelper.initialize_model(x_train.shape[1], y_train.shape[1])
model = Sequential()                                                                 
model.add(Dense(128, activation='relu', input_dim=x_train.shape[1]))                       
#model.add(Dropout(0.5))                                                             
model.add(Dense(64, activation='relu'))                                             
#model.add(Dropout(0.5))                                                             
model.add(Dense(32, activation='relu'))                                             
#model.add(Dropout(0.5))                                                             
model.add(Dense(16, activation='relu'))                                             
#model.add(Dropout(0.5))                                                             
model.add(Dense(y_train.shape[1], activation='softmax'))                                  
                                                                                     
sgd = SGD(lr=0.1, decay=1e-5, momentum=0.9, nesterov=True)                          
model.compile(loss='categorical_crossentropy',                                       
              optimizer=sgd,                                                         
              metrics=['categorical_accuracy'])#, 'fmeasure', 'precision', 'recall'])

history = model.fit(x_train, y_train,
                    epochs = 50,
                    batch_size = 64,
                    #sample_weight = df_train.weight.values,
                    validation_split = 0.20)
model.evaluate(x_test, y_test)
model.save('data/w_to_tau_DNN.h5')

In [ ]:
# get prediciton, calculate efficiencies
#l = ['score_bg', 'score_emu', 'score_etau', 'score_mutau', 'score_tautau']
l = ['score_emu', 'score_etau', 'score_mutau', 'score_tautau']
df_pred = pd.DataFrame(model.predict(x_test), columns=l)
df_pred = pd.concat([df_pred, df_test.reset_index()], axis=1)
score_corr = df_pred[l].corr()

In [ ]:
# pairplots of NN scores by categories
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
g = sns.PairGrid(df_pred[:10000], vars=l, 
                 hue='label', 
                 hue_order=['emu', 'mutau', 'etau', 'tautau'], #'bg'],
                 hue_kws=dict(marker=['o', 's', 'D', '8', '1']),
                 palette='husl',
                 size = 5
                )
g.map_diag(plt.hist, stacked=False, histtype='step', normed=False, linewidth=3)
g.map_offdiag(plt.scatter, linewidths=0.5, edgecolor="w", s=85, alpha=1)
#for i, j in zip(*np.triu_indices_from(g.axes, 1)):
#    g.axes[i, j].set_visible(False)

legend_labels = [ r'$W\rightarrow\mu, W\rightarrow e$', 
                 r'$W\rightarrow\mu, \tau\rightarrow e$', 
                 r'$\tau\rightarrow\mu, W\rightarrow e$', 
                 r'$\tau\rightarrow\mu, \tau\rightarrow e$',
                 #r'background', 
                ]
plt.legend(legend_labels, 
           bbox_to_anchor=(1.05, 1), 
           loc=2, 
           borderaxespad=0., 
           fontsize=22
          )
plt.figure(figsize=(20, 10))
plt.savefig('plots/emu_nn_scorepairs.pdf')
plt.show()

In [ ]:
# diagonal entries
labels = ['tautau', 'etau', 'mutau', 'emu']
fig, axes = plt.subplots(2, 2, figsize=(10,10))
axes[0][0].hist([df_pred[df_pred.label == n].score_emu for n in labels], 
                bins=25, 
                stacked=True, 
                histtype='stepfilled', 
                linewidth=1.5,
                alpha=0.75
               )
axes[0][0].set_yscale('log')
axes[0][0].set_xlabel(r'$e+\mu$ score')
axes[0][0].grid()

axes[0][1].hist([df_pred[df_pred.label == n].score_etau for n in labels],
                bins=25, 
                stacked=True, 
                histtype='stepfilled', 
                linewidth=1.5,
                alpha=0.75
               )
axes[0][1].legend([r'$e + \mu$', r'$\tau\rightarrow e + \mu$', r'$e + \tau\rightarrow\mu$', r'$\tau\rightarrow e + \tau\rightarrow\mu$'])
axes[0][1].set_yscale('log')
axes[0][1].set_xlabel(r'$e+\tau$ score')
axes[0][1].grid()

axes[1][0].hist([df_pred[df_pred.label == n].score_mutau for n in labels], 
                bins=25, 
                stacked=True, 
                histtype='stepfilled', 
                linewidth=1.5,
                alpha=0.75
               )
axes[1][0].set_yscale('log')
axes[1][0].set_xlabel(r'$\mu+\tau$ score')
axes[1][0].grid()

axes[1][1].hist([df_pred[df_pred.label == n].score_tautau for n in labels], 
                bins=25, 
                stacked=True, 
                histtype='stepfilled', 
                linewidth=1.5, 
                alpha=0.75
               )
axes[1][1].set_yscale('log')
axes[1][1].set_xlabel(r'$\tau+\tau$ score')
axes[1][1].grid()

plt.show()

In [ ]:
# diagonal entries
labels = ['tautau', 'etau', 'mutau', 'emu']
fig, axes = plt.subplots(2, 2, figsize=(10,10))
axes[0][0].hist([df_pred[df_pred.label == n].score_emu for n in labels], 
                bins=25, 
                normed=True, 
                histtype='step', 
                linewidth=1.5,
                alpha=0.75
               )
axes[0][0].set_yscale('log')
axes[0][0].set_xlabel(r'$e+\mu$ score')
axes[0][0].grid()

axes[0][1].hist([df_pred[df_pred.label == n].score_etau for n in labels],
                bins=25, 
                normed=True, 
                histtype='step', 
                linewidth=1.5,
                alpha=0.75
               )
axes[0][1].legend([r'$e + \mu$', r'$\tau\rightarrow e + \mu$', r'$e + \tau\rightarrow\mu$', r'$\tau\rightarrow e + \tau\rightarrow\mu$'])
axes[0][1].set_yscale('log')
axes[0][1].set_xlabel(r'$e+\tau$ score')
axes[0][1].grid()

axes[1][0].hist([df_pred[df_pred.label == n].score_mutau for n in labels], 
                bins=25, 
                normed=True, 
                histtype='step', 
                linewidth=1.5,
                alpha=0.75
               )
axes[1][0].set_yscale('log')
axes[1][0].set_xlabel(r'$\mu+\tau$ score')
axes[1][0].grid()

axes[1][1].hist([df_pred[df_pred.label == n].score_tautau for n in labels], 
                bins=25, 
                normed=True, 
                histtype='step', 
                linewidth=1.5, 
                alpha=0.75
               )
axes[1][1].set_yscale('log')
axes[1][1].set_xlabel(r'$\tau+\tau$ score')
axes[1][1].grid()

plt.show()